# Part IV — Professional Django  
## 18. Testing (Unit, Integration, E2E) — Industry-Grade Strategy + Hands-On

This chapter turns “I have some tests” into “I can ship changes confidently.”

You’ll learn how professional Django teams test:

- **Unit tests** for pure logic (fast, isolated, tons of coverage)
- **Integration tests** for Django views/ORM/forms (the backbone of web apps)
- **End-to-end (E2E) tests** for critical user journeys (few, expensive, high value)

You’ll also learn how to keep tests:
- readable
- fast
- deterministic
- resistant to refactors
- useful for preventing regressions (including performance regressions like N+1)

---

## 18.0 Learning Outcomes

By the end, you should be able to:

1. Explain the **test pyramid** and apply it to Django projects.
2. Choose what to test at each level: unit vs integration vs E2E.
3. Write robust Django tests using:
   - `TestCase` + Django test client
   - pytest + pytest-django (recommended in many teams)
4. Use factories (factory_boy) instead of brittle fixtures.
5. Mock external services (email, HTTP APIs) safely.
6. Test authorization and scoping (multi-tenant/org apps) correctly.
7. Prevent performance regressions using query-count tests (`assertNumQueries`).
8. Run tests like CI does (lint + format check + tests + coverage).

---

## 18.1 The Test Pyramid (How Many Tests of Each Type)

### 18.1.1 The pyramid
- **Unit tests** (many): pure functions/classes, no DB, no Django request cycle
- **Integration tests** (some): Django views/forms/models/ORM, DB used
- **E2E tests** (few): real browser flows or near-real flows

Why this is standard:
- Unit tests are fast and pinpoint failures.
- Integration tests prove framework wiring works.
- E2E tests catch “it breaks in the browser” issues but are slow and flaky if overused.

### 18.1.2 What you should NOT test (common waste)
- “Django renders a template correctly” at pixel level
- third-party library internals (unless you wrap them)
- simple boilerplate lines
- admin UI details (unless you added critical custom logic)

You test *your behavior*, not Django’s behavior.

---

## 18.2 Django’s Built-in Test System vs pytest (Industry Reality)

You can test Django in two popular ways:

### Option A: Django `TestCase` (unittest style)
- Comes with Django, no extra packages
- Great and widely used
- A bit more verbose

### Option B (common in industry): pytest + pytest-django
- Cleaner syntax
- Powerful fixtures
- Better parametrization
- Huge ecosystem

Many teams use pytest for everything.

This chapter shows both, but strongly encourages pytest-django once you’re comfortable.

---

## 18.3 Baseline Test Rules (Make Tests Reliable)

### Rule 1: Tests must be deterministic
No randomness without seeding, no reliance on current time unless frozen, no external network.

### Rule 2: Each test should arrange-act-assert
- Arrange: create data
- Act: call function/view
- Assert: check results

### Rule 3: Avoid “magic global state”
If tests pass alone but fail when run together, you likely have hidden state.

### Rule 4: Keep tests readable
A test is documentation. If it’s unreadable, it won’t be maintained.

---

## 18.4 Project Test Structure (Recommended Layout)

Two common patterns:

### Pattern 1: tests inside apps (Django default)
- `articles/tests.py`
- `tasks/tests.py`

Pros: close to code.  
Cons: can become large messy files.

### Pattern 2 (common in bigger projects): separate `tests/` package
Example:

```text
tests/
  test_articles_views.py
  test_tasks_permissions.py
  factories.py
  conftest.py
```

Pros: easy to organize by behavior.  
Cons: need discipline to keep mapping clear.

Either is fine. For “industry standard,” many teams prefer a `tests/` directory plus
factories in one place.

---

## 18.5 Install the “Professional Testing Stack” (Recommended)

If you’re staying with Django TestCase only, you can skip this section. If you want
a modern setup, install:

- `pytest`
- `pytest-django`
- `factory-boy`
- `coverage` (or `pytest-cov`)
- (optional) `freezegun` for freezing time
- (optional) `responses` for mocking HTTP requests

### 18.5.1 Install packages
Add to `requirements-dev.txt`:

```text
pytest
pytest-django
pytest-cov
factory-boy
freezegun
responses
```

Install:

```bash
python -m pip install -r requirements-dev.txt
```

---

## 18.6 Configure pytest-django

### 18.6.1 Create `pytest.ini`
At repo root, create `pytest.ini`:

```ini
[pytest]
DJANGO_SETTINGS_MODULE = config.settings
python_files = tests.py test_*.py *_tests.py
addopts = -q
```

Explanation:
- `DJANGO_SETTINGS_MODULE` tells pytest how to load Django.
- `python_files` defines test file patterns.
- `addopts = -q` makes output less noisy (optional).

### 18.6.2 Run pytest
```bash
python -m pytest
```

If you already have Django `TestCase` tests, pytest will run them too.

---

## 18.7 Factories (Stop Hand-Writing Data Setup)

Factories generate valid model instances with minimal code.

### 18.7.1 Why factories are better than fixtures
- fixtures become stale as models evolve
- fixtures are hard to read and customize per test
- factories create exactly what the test needs

### 18.7.2 Create `tests/factories.py`
Create a `tests/` folder with `__init__.py`:

```bash
mkdir -p tests
touch tests/__init__.py
```

Now create `tests/factories.py`:

```python
import factory
from django.contrib.auth import get_user_model
from django.utils import timezone

from articles.models import Article, Tag
from orgs.models import Membership, Organization
from tasks.models import Task


class UserFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = get_user_model()

    username = factory.Sequence(lambda n: f"user{n}")
    email = factory.LazyAttribute(lambda o: f"{o.username}@example.com")
    password = factory.PostGenerationMethodCall("set_password", "pass12345")


class OrganizationFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Organization

    name = factory.Sequence(lambda n: f"Org {n}")
    slug = factory.Sequence(lambda n: f"org-{n}")


class MembershipFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Membership

    organization = factory.SubFactory(OrganizationFactory)
    user = factory.SubFactory(UserFactory)
    role = Membership.Role.MEMBER


class TagFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Tag

    name = factory.Sequence(lambda n: f"Tag {n}")
    slug = factory.Sequence(lambda n: f"tag-{n}")


class ArticleFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Article

    title = factory.Sequence(lambda n: f"Article {n}")
    slug = factory.Sequence(lambda n: f"article-{n}")
    body = "Body " * 20
    status = Article.Status.DRAFT
    published_at = None
    author = factory.SubFactory(UserFactory)

    @factory.post_generation
    def tags(self, create, extracted, **kwargs):
        if not create:
            return
        if extracted:
            for t in extracted:
                self.tags.add(t)

    @factory.lazy_attribute
    def created_at(self):
        return timezone.now()


class TaskFactory(factory.django.DjangoModelFactory):
    class Meta:
        model = Task

    organization = factory.SubFactory(OrganizationFactory)
    title = factory.Sequence(lambda n: f"Task {n}")
    description = "desc"
    status = Task.Status.OPEN
    priority = Task.Priority.MEDIUM
    due_date = None

    assigned_to = factory.SubFactory(UserFactory)
    created_by = factory.SubFactory(UserFactory)
    updated_by = factory.SelfAttribute("created_by")
```

#### Explanation: key factory concepts
- `factory.Sequence`: ensures uniqueness (`user0`, `user1`, …).
- `PostGenerationMethodCall("set_password", ...)`: stores hashed passwords.
- `SubFactory`: creates required related objects automatically.
- `post_generation` for M2M tags: lets you do:
  - `ArticleFactory(tags=[tag1, tag2])`

---

## 18.8 pytest Fixtures for Authenticated Clients (Clean Tests)

Create `tests/conftest.py`:

```python
import pytest

from tests.factories import UserFactory


@pytest.fixture
def user(db):
    return UserFactory()


@pytest.fixture
def auth_client(client, user):
    client.login(username=user.username, password="pass12345")
    return client
```

Explanation:
- `db` fixture enables DB access in pytest.
- `client` is pytest-django’s Django test client fixture.
- `auth_client` gives you a logged-in client easily.

---

## 18.9 Unit Tests (Pure Logic) — Fast, No DB

### 18.9.1 Example: test a query-param parser utility (pure function)
Suppose you create `core/utils.py`:

```python
def clamp_page_size(value: int, *, default: int = 20, max_size: int = 100) -> int:
    if value is None:
        return default
    if value < 1:
        return default
    return min(value, max_size)
```

Unit test in `tests/test_utils.py`:

```python
from core.utils import clamp_page_size


def test_clamp_page_size_defaults():
    assert clamp_page_size(None) == 20
    assert clamp_page_size(0) == 20


def test_clamp_page_size_caps():
    assert clamp_page_size(10) == 10
    assert clamp_page_size(999) == 100
```

Why this is valuable:
- runs in milliseconds
- failure points directly to logic
- avoids DB overhead for pure rules

---

## 18.10 Integration Tests (Django Views + ORM + Templates)

These are your bread-and-butter tests for Django apps.

### 18.10.1 Articles: list view shows only published
`tests/test_articles_public.py`:

```python
from django.urls import reverse
from django.utils import timezone

from tests.factories import ArticleFactory


def test_articles_list_shows_published_only(client):
    ArticleFactory(status="draft")
    ArticleFactory(status="published", published_at=timezone.now())

    url = reverse("articles:list")
    response = client.get(url)

    assert response.status_code == 200
    content = response.content.decode("utf-8")
    assert "published" in content.lower()
```

Better: assert titles to avoid brittle string checks:

```python
def test_articles_list_titles(client):
    a1 = ArticleFactory(
        title="Visible",
        status="published",
        published_at=timezone.now(),
    )
    ArticleFactory(title="Hidden", status="draft")

    response = client.get(reverse("articles:list"))
    assert response.status_code == 200
    assert "Visible" in response.content.decode("utf-8")
    assert "Hidden" not in response.content.decode("utf-8")
```

### 18.10.2 Draft visibility: anonymous gets 404, author sees it
```python
from django.urls import reverse

from tests.factories import ArticleFactory


def test_draft_hidden_from_anonymous(client):
    article = ArticleFactory(status="draft")
    url = reverse("articles:detail", kwargs={"slug": article.slug})
    response = client.get(url)
    assert response.status_code == 404


def test_draft_visible_to_author(auth_client, user):
    article = ArticleFactory(status="draft", author=user)
    url = reverse("articles:detail", kwargs={"slug": article.slug})
    response = auth_client.get(url)
    assert response.status_code == 200
```

#### Why these are “integration”
They test:
- URL routing
- view logic
- ORM queries
- template rendering
- middleware/auth effects

---

## 18.11 Authorization Tests (The Highest Value in Multi-User Apps)

For Project 2 (orgs/tasks), most expensive bugs are scoping and permissions.

### 18.11.1 Non-member cannot access org task list
`tests/test_tasks_scoping.py`:

```python
from django.urls import reverse

from tests.factories import MembershipFactory, OrganizationFactory, UserFactory


def test_non_member_cannot_access_org(client):
    org = OrganizationFactory(slug="acme")
    member = UserFactory(username="member")
    MembershipFactory(organization=org, user=member)

    outsider = UserFactory(username="outsider")
    client.login(username=outsider.username, password="pass12345")

    url = reverse("tasks:list", kwargs={"org_slug": "acme"})
    response = client.get(url)
    assert response.status_code == 404
```

### 18.11.2 Member cannot export CSV
```python
from django.urls import reverse

from orgs.models import Membership
from tests.factories import MembershipFactory, OrganizationFactory, UserFactory


def test_member_cannot_export_csv(client):
    org = OrganizationFactory(slug="acme")
    bob = UserFactory(username="bob")
    MembershipFactory(organization=org, user=bob, role=Membership.Role.MEMBER)

    client.login(username="bob", password="pass12345")

    url = reverse("tasks:export_csv", kwargs={"org_slug": "acme"})
    response = client.get(url)
    assert response.status_code == 403
```

### 18.11.3 Admin can export CSV
```python
from django.urls import reverse

from orgs.models import Membership
from tests.factories import MembershipFactory, OrganizationFactory, UserFactory


def test_admin_can_export_csv(client):
    org = OrganizationFactory(slug="acme")
    alice = UserFactory(username="alice")
    MembershipFactory(organization=org, user=alice, role=Membership.Role.ADMIN)

    client.login(username="alice", password="pass12345")

    url = reverse("tasks:export_csv", kwargs={"org_slug": "acme"})
    response = client.get(url)
    assert response.status_code == 200
    assert response["Content-Type"] == "text/csv"
    assert "attachment" in response["Content-Disposition"]
```

These tests prevent tenant data leaks and privilege escalation.

---

## 18.12 Form Tests (Validation Rules Without Views)

Forms are ideal unit-ish tests (fast) but can still hit DB for ModelForm.

### 18.12.1 Test ArticleForm “publish sets published_at”
```python
from django.utils import timezone

from articles.forms import ArticleForm
from articles.models import Article
from tests.factories import UserFactory


def test_article_form_sets_published_at(db):
    user = UserFactory()
    form = ArticleForm(
        data={
            "title": "T",
            "slug": "t",
            "body": "Body " * 10,
            "status": Article.Status.PUBLISHED,
            "published_at": "",
            "tags": [],
        }
    )
    assert form.is_valid()

    article = form.save(commit=False)
    article.author = user
    article.save()
    form.save_m2m()

    assert article.published_at is not None
    assert article.status == Article.Status.PUBLISHED
```

---

## 18.13 Email Tests (No Real Sending)

### 18.13.1 Test “publish transition sends one email”
If your code sends email in the edit view and uses Django’s test backend, use outbox:

```python
from django.core import mail
from django.urls import reverse
from django.utils import timezone

from articles.models import Article
from tests.factories import ArticleFactory, UserFactory


def test_publish_transition_sends_email(client, db):
    user = UserFactory(username="u1", email="u1@example.com")
    client.login(username="u1", password="pass12345")

    article = ArticleFactory(status=Article.Status.DRAFT, author=user)

    url = reverse("articles:edit", kwargs={"slug": article.slug})
    response = client.post(
        url,
        data={
            "title": article.title,
            "slug": article.slug,
            "body": article.body,
            "status": Article.Status.PUBLISHED,
            "published_at": timezone.now(),
            "tags": [],
        },
    )

    assert response.status_code == 302
    assert len(mail.outbox) == 1
    assert mail.outbox[0].to == ["u1@example.com"]
```

---

## 18.14 Mocking External Services (HTTP APIs) Safely

Never let tests call real network.

### 18.14.1 Example service function that calls an external API
`core/external.py`:

```python
import requests


def fetch_exchange_rate(base: str, quote: str) -> float:
    r = requests.get(
        "https://api.example.com/rates",
        params={"base": base, "quote": quote},
        timeout=5,
    )
    r.raise_for_status()
    payload = r.json()
    return float(payload["rate"])
```

### 18.14.2 Test with `responses` (no network)
`tests/test_external.py`:

```python
import responses

from core.external import fetch_exchange_rate


@responses.activate
def test_fetch_exchange_rate():
    responses.add(
        method=responses.GET,
        url="https://api.example.com/rates",
        json={"rate": "1.23"},
        status=200,
    )

    rate = fetch_exchange_rate("USD", "EUR")
    assert rate == 1.23
```

Key benefits:
- deterministic
- fast
- no flaky external dependencies

---

## 18.15 Performance Regression Tests (Prevent N+1)

You already used `assertNumQueries` in Django TestCase. You can still do it with
pytest by using Django’s TestCase style or fixtures.

### 18.15.1 Use `django_assert_num_queries` (pytest-django)
pytest-django provides a fixture:

```python
from django.urls import reverse
from django.utils import timezone

from articles.models import Article
from tests.factories import ArticleFactory, TagFactory


def test_articles_list_avoids_n_plus_one(client, django_assert_num_queries, db):
    tag = TagFactory(slug="django")

    for i in range(20):
        a = ArticleFactory(
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
        )
        a.tags.add(tag)

    url = reverse("articles:list")

    with django_assert_num_queries(3):
        # Typical: 1 for articles + 1 for tags prefetch + maybe 1 for session/auth
        response = client.get(url)
        assert response.status_code == 200
```

Important:
- exact number can vary based on middleware and template context.
- aim for “small and stable,” not “exactly 2 everywhere” unless you control it tightly.

### 18.15.2 Alternative: measure query count by focusing on view queryset logic
Another professional pattern is to unit-test query composition and separately ensure
views use prefetch/select_related.

---

## 18.16 Time Freezing (Stop “Tests fail sometimes”)

Time-sensitive logic (published_at, due_date) can be flaky.

### 18.16.1 Freeze time with freezegun
```python
from freezegun import freeze_time
from django.utils import timezone


@freeze_time("2026-02-04 12:00:00")
def test_freeze_time():
    assert timezone.now().isoformat().startswith("2026-02-04T12:00:00")
```

Use it when you compare timestamps or generate date-based filenames.

---

## 18.17 End-to-End Testing (E2E): When and How (Practical Guidance)

E2E tests simulate real user flows:
- login → create task → edit → export
- publish article → verify email content exists (maybe via test backend)
- comment submission → moderation

### 18.17.1 Why you only write a few E2E tests
- slower
- more brittle (depends on full stack)
- harder to debug

### 18.17.2 Recommended E2E tool (industry common)
Many teams use Playwright or Selenium. A solid modern choice is Playwright.

At this stage, treat E2E as optional; integration tests cover most behavior in
Django apps.

If you want E2E now, you can:
- run Django live server
- drive a headless browser
- verify critical flows

We can implement this later in the “Advanced Testing” chapter as well.

---

## 18.18 Coverage (Make It Useful, Not Vanity)

Coverage answers: “Which lines executed?”  
It does *not* guarantee correctness.

But it’s still useful to:
- detect untested modules
- enforce minimum standards

### 18.18.1 Run tests with coverage (pytest-cov)
```bash
python -m pytest --cov=. --cov-report=term-missing
```

Interpretation:
- focus on important modules (business logic, permissions, services)
- don’t chase 100% blindly

Industry baseline:
- 70–90% depending on project maturity and risk level
- higher for critical systems

---

## 18.19 CI-Style Test Command (What You Should Run Locally)

A common “local CI mimic”:

```bash
python -m ruff check .
python -m black . --check
python -m pytest --cov=. --cov-report=term-missing
```

This is what your pipeline will eventually run.

---

# 18.20 LAB: Upgrade Your Project’s Test Suite (Step-by-Step)

## Lab A — Install pytest-django and move one test to pytest style
1. Add pytest packages to `requirements-dev.txt`, install.
2. Create `pytest.ini`.
3. Create `tests/test_articles_visibility.py` with:
   - draft hidden to anonymous
   - draft visible to author
4. Run `python -m pytest`.

## Lab B — Introduce factories and refactor tests
1. Add `factory-boy`.
2. Create `tests/factories.py`.
3. Replace manual `User.objects.create_user(...)` in at least 3 tests with factories.
4. Ensure tests remain readable (don’t over-abstract).

## Lab C — Add a permission/scoping regression test for tasks
Write tests that ensure:
- outsider gets 404 on `/orgs/<slug>/tasks/`
- member gets 403 on export
- admin gets 200 on export

## Lab D — Add one query-count test to prevent N+1
Pick either:
- article list tags rendering, or
- task list with assigned_to/created_by showing

Use `django_assert_num_queries` or Django TestCase `assertNumQueries`.

---

## 18.21 Exercises (Do These Before Proceeding)

1. Write one **unit test** for a pure function you create (e.g., parsing or business rule).
2. Write two **integration tests**:
   - one for a view success path
   - one for a view forbidden path (403 or 404)
3. Add factories for any model you currently create manually in >3 places.
4. Add a test that mocks an external HTTP request using `responses`.
5. Add coverage command to your README and document how to run tests.

---

## 18.22 Chapter Summary (What You Should Retain)

- Use the test pyramid: lots of unit tests, some integration, few E2E.
- pytest-django + factory-boy is a common professional Django stack.
- Test the highest-risk areas:
  - authorization/scoping
  - data integrity constraints
  - critical workflows (publish, export)
- Prevent performance regressions with query-count tests.
- Mock all external systems (HTTP, email providers) in tests.
- Make your local run match CI.

---

Next chapter: **Part IV — 19. Security (OWASP-Aligned)**  
We’ll harden settings, understand CSRF/XSS/SQL injection in Django-specific terms,
add security headers, secure uploads, and build a security review checklist you can
apply to any Django app.